In [1]:
from keras.models import *
from keras.layers import *
import numpy as np
import pandas as pd

series_size = 7
feature_size = 2

Using TensorFlow backend.


In [2]:
dataframe = pd.read_csv('dorsey_2000.csv')
dataframe = dataframe.dropna()
del dataframe['Date']
dataframe["Change In Soil Moisture Percent -8in (pct)"] = np.where(dataframe["Change In Soil Moisture Percent -8in (pct)"] >= 0, 0, dataframe["Change In Soil Moisture Percent -8in (pct)"])
dataframe["Change In Soil Moisture Percent -8in (pct)"] = np.where(dataframe["Change In Soil Moisture Percent -8in (pct)"] < 0, 1, dataframe["Change In Soil Moisture Percent -8in (pct)"])
dataframe

,Air Temperature Observed (degF),Precipitation Increment - Snow-adj (in),Change In Precipitation Accumulation (in),Change In Soil Moisture Percent -8in (pct)
1539,44.0,0.0,0.0,0.0
1540,33.0,0.0,0.0,0.0
1541,48.0,0.0,0.0,0.0
1542,42.0,0.0,0.0,0.0
1543,39.0,0.0,0.0,0.0
1544,38.0,0.0,0.0,0.0
1545,31.0,0.2,0.0,0.0
1546,29.0,0.2,0.2,0.0
1547,24.0,0.1,0.1,1.0
1548,22.0,0.2,0.0,1.0


In [3]:
npdata = np.asarray(dataframe)
npdata.shape

(5522, 4)

In [4]:
time_models_x = []
time_models_y = []
for i in range(0, npdata.shape[0]-series_size):
    curr_time_x = npdata[i:i+series_size, :feature_size]
    time_models_x.append(curr_time_x)
    curr_time_y = npdata[i+series_size-1, feature_size]
    time_models_y.append(curr_time_y)
time_models_x = np.asarray(time_models_x)
time_models_y = np.asarray(time_models_y)
print(time_models_x)
print(time_models_y)

[[[44.   0. ]
  [33.   0. ]
  [48.   0. ]
  ...
  [39.   0. ]
  [38.   0. ]
  [31.   0.2]]

 [[33.   0. ]
  [48.   0. ]
  [42.   0. ]
  ...
  [38.   0. ]
  [31.   0.2]
  [29.   0.2]]

 [[48.   0. ]
  [42.   0. ]
  [39.   0. ]
  ...
  [31.   0.2]
  [29.   0.2]
  [24.   0.1]]

 ...

 [[34.   0. ]
  [33.   0. ]
  [38.   0. ]
  ...
  [42.   0. ]
  [39.   0. ]
  [36.   0. ]]

 [[33.   0. ]
  [38.   0. ]
  [37.   0.1]
  ...
  [39.   0. ]
  [36.   0. ]
  [35.   0. ]]

 [[38.   0. ]
  [37.   0.1]
  [42.   0. ]
  ...
  [36.   0. ]
  [35.   0. ]
  [38.   0. ]]]
[0.  0.2 0.1 ... 0.  0.  0. ]


In [5]:
model = Sequential()
model.add(LSTM(16, input_shape=(series_size,feature_size), return_sequences=False))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 16)                1216      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 1,233
Trainable params: 1,233
Non-trainable params: 0
_________________________________________________________________


In [6]:
history = model.fit(time_models_x, time_models_y, batch_size=256, epochs=35, validation_split=0.1, shuffle=True)
model.save("lstm.h5")

Train on 4963 samples, validate on 552 samples
Epoch 1/35
4963/4963 [==============================] - 2s 431us/step - loss: 0.7957 - acc: 0.0647 - val_loss: 0.7159 - val_acc: 0.2627
Epoch 2/35
4963/4963 [==============================] - 0s 45us/step - loss: 0.6387 - acc: 0.5882 - val_loss: 0.6038 - val_acc: 0.7192
Epoch 3/35
4963/4963 [==============================] - 0s 46us/step - loss: 0.5410 - acc: 0.7344 - val_loss: 0.5171 - val_acc: 0.7192
Epoch 4/35
4963/4963 [==============================] - 0s 45us/step - loss: 0.4501 - acc: 0.7346 - val_loss: 0.4164 - val_acc: 0.7192
Epoch 5/35
4963/4963 [==============================] - 0s 47us/step - loss: 0.3277 - acc: 0.7346 - val_loss: 0.3068 - val_acc: 0.7192
Epoch 6/35
4963/4963 [==============================] - 0s 45us/step - loss: 0.1957 - acc: 0.7346 - val_loss: 0.2567 - val_acc: 0.7192
Epoch 7/35
4963/4963 [==============================] - 0s 46us/step - loss: 0.1430 - acc: 0.7346 - val_loss: 0.2401 - val_acc: 0.7192
Epoch 8

In [7]:
test_series = [np.array([25, 0]), np.array([12,0.2]), np.array([11, 0.2]), np.array([18, 0.6]), np.array([18, 0]), np.array([3, 0.1]), np.array([25, 0])]
test_series = np.array(test_series)
test_series = test_series.reshape((1, test_series.shape[0], test_series.shape[1]))
test_series.shape

(1, 7, 2)

In [8]:
model.predict(test_series)

array([[0.11294386]], dtype=float32)